# Create a FABRIC Facility Port

This notebook shows how to use create an facility port to connect your FABRIC experiment to an external facility. 



## Import the FABlib Library

In [1]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                     
fablib.show_config();

Credential Manager,cm.fabric-testbed.net
Orchestrator,orchestrator.fabric-testbed.net
Project ID,990d8a8b-7e50-4d13-a3be-0f133ffa8653
Token File,/home/fabric/.tokens.json
Bastion Host,bastion-renc-1.fabric-testbed.net
Bastion Username,mcevik_0000100641
Bastion Private Key File,/home/fabric/work/fabric_config/mcevik_fabric_bastion_1
Slice Private Key File,/home/fabric/work/fabric_config/mcevik_fabric_sliver_1
Slice Public Key File,/home/fabric/work/fabric_config/mcevik_fabric_sliver_1.pub
Log File,/tmp/fablib/fablib.log
Log Level,INFO


## Create the Experiment Slice

The following creates a single node with basic compute capabilities. You build a slice by creating a new slice and adding resources to the slice. After you build the slice, you must submit a request for the slice to be instantiated.   

By default, the submit function will block until the node is ready and will display the progress of your slice being built.



In [2]:
slice_name = "Slice-AWSDX"

facility_port_site='FIU'

### Facility port info

#### Find the facility ports on the site
List the facility ports
- Facility Port Name
- Available VLAN range
- Currently allocated VLAN range

In [3]:
cell_output = 'pandas' # 'json'
output_list = fablib.list_facility_ports(filter_function=lambda x: x['site_name'] == facility_port_site, output=cell_output)

Name,Site,Interface Name,VLAN Range,Allocated VLAN Range,Local Name,Device Name,Region
AmLight-EXP-Layer2-FIU,FIU,port+fiu-data-sw:HundredGigE0/0/0/23:facility+AmLight-EXP-Layer2-FIU-int,['4000-4019'],N/A,N/A,N/A,N/A
AmLight-Layer3-FIU,FIU,port+fiu-data-sw:HundredGigE0/0/0/23:facility+AmLight-Layer3-FIU-int,['3001-3008'],N/A,N/A,N/A,N/A


#### Select the Facility Port on the site

Set the facility port name and select the VLAN to use

In [4]:
facility_port_name = 'AmLight-EXP-Layer2-FIU' # VLAN Range: 4000-4019
#facility_port_name ='AmLight-Layer3-FIU'     # VLAN Range: 3001-3008

facility_port_vlan='4015'

### Create Slice

Create new_slice and add node(s) to the FABRIC topology

In [5]:
slice = fablib.new_slice(name=slice_name)

# Example: One VM on FABRIC-STAR with a NIC - sharedNIC (basic) or dedicated smartNIC (ConnectX_6)
node = slice.add_node(name=f"Node1", site='STAR')

#node_iface = node.add_component(model='NIC_ConnectX_6', name="nic1").get_interfaces()[0]
node_iface = node.add_component(model='NIC_Basic', name="nic1").get_interfaces()[0]

Add facility port to the network service configuration along with the node interface(s)

In [6]:
facility_port = slice.add_facility_port(name=facility_port_name, site=facility_port_site, vlan=facility_port_vlan)
facility_port_interface =facility_port.get_interfaces()[0]

#net = slice.add_l2network(name=f'net_facility_port', interfaces=[node_iface,facility_port_interface])
net = slice.add_l2network(name=f'net_facility_port', interfaces=[])
net.add_interface(node_iface)
net.add_interface(facility_port_interface)

In [14]:
print(f"facility_port.get_site(): {facility_port.get_site()}")
print(f"facility_port.get_fim_interface(): {facility_port.get_fim_interface()}")

facility_port.get_site(): FIU
facility_port.get_fim_interface(): {'node_id': 'e1887f1e-a860-4b6f-a80b-dfe3a1970a85', 'name': 'AmLight-EXP-Layer2-FIU', 'site': 'FIU', 'type': 'Facility'}


Submit slice request to create

In [15]:
slice.submit();


Retry: 9, Time: 269 sec


ID,ac215229-873c-4107-b4d3-96ed9a90a107
Name,Slice-AWSDX
Lease Expiration (UTC),2024-10-04 17:28:55 +0000
Lease Start (UTC),2024-10-03 17:28:55 +0000
Project ID,990d8a8b-7e50-4d13-a3be-0f133ffa8653
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
813b6a8a-9a43-49b6-afb6-1640f4dd5256,Node1,2,8,10,default_rocky_8,qcow2,star-w6.fabric-testbed.net,STAR,rocky,2001:400:a100:3030:f816:3eff:fe1c:34eb,Active,,ssh -i /home/fabric/work/fabric_config/mcevik_fabric_sliver_1 -F /home/fabric/work/fabric_config/ssh_config rocky@2001:400:a100:3030:f816:3eff:fe1c:34eb,/home/fabric/work/fabric_config/mcevik_fabric_sliver_1.pub,/home/fabric/work/fabric_config/mcevik_fabric_sliver_1


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
0204e3e2-c4ba-4996-9706-0620a1f0d622,net_facility_port,L2,L2STS,None,None,None,Active,


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node,Switch Port
Node1-nic1-p1,p1,Node1,net_facility_port,100,config,,02:A9:BF:EC:7F:C7,eth1,eth1,fe80::27e1:577f:6571:6ef5,4,HundredGigE0/0/0/3



Time to print interfaces 273 seconds


## Observe the Slice's Attributes


In [16]:
slice = fablib.get_slice(slice_name)

In [17]:
slice.show()
slice.list_nodes()
slice.list_networks()
slice.list_interfaces()


ID,ac215229-873c-4107-b4d3-96ed9a90a107
Name,Slice-AWSDX
Lease Expiration (UTC),2024-10-04 17:28:55 +0000
Lease Start (UTC),2024-10-03 17:28:55 +0000
Project ID,990d8a8b-7e50-4d13-a3be-0f133ffa8653
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
813b6a8a-9a43-49b6-afb6-1640f4dd5256,Node1,2,8,10,default_rocky_8,qcow2,star-w6.fabric-testbed.net,STAR,rocky,2001:400:a100:3030:f816:3eff:fe1c:34eb,Active,,ssh -i /home/fabric/work/fabric_config/mcevik_fabric_sliver_1 -F /home/fabric/work/fabric_config/ssh_config rocky@2001:400:a100:3030:f816:3eff:fe1c:34eb,/home/fabric/work/fabric_config/mcevik_fabric_sliver_1.pub,/home/fabric/work/fabric_config/mcevik_fabric_sliver_1


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
0204e3e2-c4ba-4996-9706-0620a1f0d622,net_facility_port,L2,L2STS,None,None,None,Active,


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node,Switch Port
Node1-nic1-p1,p1,Node1,net_facility_port,100,config,,02:A9:BF:EC:7F:C7,eth1,eth1,fe80::27e1:577f:6571:6ef5,4,HundredGigE0/0/0/3


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node,Switch Port
Node1-nic1-p1,p1,Node1,net_facility_port,100,config,,02:A9:BF:EC:7F:C7,eth1,eth1,fe80::27e1:577f:6571:6ef5,4,HundredGigE0/0/0/3


## Run the Experiment

Most experiments will require automated configuration and execution. You can use the fablib library to execute arbitrary commands on your node. 

The following code demonstrates how to use fablib to execute a "Hello, FABRIC" bash script. The library uses the bastion and VM keys defined at the top of this notebook to jump through the bastion host and execute the script.

In [18]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

# Configure the interface(s) of the VM(s) with designated subnet
subnet = IPv4Network("192.168.1.0/24")
available_ips = list(subnet)[2:]

In [19]:
node1 = slice.get_node(name=f"Node1")        
node1_iface = node1.get_interface(network_name=f'net_facility_port') 
node1_addr = available_ips.pop(99)
print(f"node1_addr: {node1_addr}")
node1_iface.ip_addr_add(addr=node1_addr, subnet=subnet)

stdout, stderr = node1.execute(f'ip addr show {node1_iface.get_os_interface()}')

stdout, stderr = node1.execute(f'sudo ip link set dev {node1_iface.get_physical_os_interface_name()} up')

stdout, stderr = node1.execute(f'sudo ip link set dev {node1_iface.get_os_interface()} up')

node1_addr: 192.168.1.101
3: eth1: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether 02:a9:bf:ec:7f:c7 brd ff:ff:ff:ff:ff:ff
    altname enp7s0
    inet 192.168.1.101/24 scope global eth1
       valid_lft forever preferred_lft forever
    inet6 fe80::27e1:577f:6571:6ef5/64 scope link noprefixroute 
       valid_lft forever preferred_lft forever


In [ ]:
# Ping the node(s) that are active on the Facility Port side (e.g 192.168.1.10)
node1 = slice.get_node(name=f"Node1")     
node1_iface = node1.get_interface(network_name=f'net_facility_port') 

stdout, stderr = node1.execute(f'ping -c 5 192.168.1.10')

## Delete the Slice

Please delete your slice when you are done with your experiment.

In [ ]:
slice.delete()